# WX-Extreme Basic Usage Examples

This notebook demonstrates how to use the WX-Extreme package for analyzing extreme weather events and evaluating weather predictions.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from wx_extreme.utils import spatial_utils, met_utils, stat_utils, plot_utils

## 1. Load Sample Data

First, let's create some sample weather data using xarray.

In [ ]:
# Create sample coordinates
lat = np.linspace(30, 50, 50)
lon = np.linspace(-10, 20, 60)
time = pd.date_range('2023-01-01', '2023-12-31', freq='D')

# Create sample temperature data
temp = 15 + 10 * np.cos(np.pi * (time.dayofyear - 182) / 182)[:, np.newaxis, np.newaxis] + \
       5 * np.cos(np.pi * (lat - 40) / 20)[np.newaxis, :, np.newaxis] + \
       2 * np.random.randn(len(time), len(lat), len(lon))

# Create xarray dataset
ds = xr.Dataset(
    data_vars=dict(
        temperature=(['time', 'latitude', 'longitude'], temp),
    ),
    coords=dict(
        time=time,
        latitude=lat,
        longitude=lon
    )
)

ds

## 2. Spatial Analysis

Let's calculate grid spacing and areas.

In [ ]:
# Calculate grid spacing
dx, dy = spatial_utils.get_grid_spacing(ds.latitude, ds.longitude)
print(f"Grid spacing: dx = {dx:.2f} km, dy = {dy:.2f} km")

# Calculate grid cell areas
areas = spatial_utils.calculate_area(ds.latitude, ds.longitude)
print(f"\nGrid cell areas range: {areas.min().item():.2f} to {areas.max().item():.2f} km²")

## 3. Statistical Analysis

Let's analyze temperature extremes.

In [ ]:
# Calculate 95th percentile of temperature
temp_95 = stat_utils.calculate_percentile(ds.temperature, q=95, dim='time')

# Calculate exceedance probability
exceed_prob = stat_utils.exceedance_probability(ds.temperature, threshold=25, dim='time')

# Fit extreme value distribution at a specific location
temp_series = ds.temperature.sel(latitude=40, longitude=0, method='nearest')
params, info = stat_utils.extreme_value_fit(temp_series, distribution='gev')

print("GEV Distribution Parameters:")
print(f"Shape: {params[0]:.3f}")
print(f"Location: {params[1]:.3f}")
print(f"Scale: {params[2]:.3f}")

## 4. Visualization

Let's create some plots to visualize our results.

In [ ]:
# Create a figure with two subplots
fig = plt.figure(figsize=(15, 6))

# Plot mean temperature
ax1 = plt.subplot(121, projection=ccrs.PlateCarree())
plot_utils.plot_field(
    ds.temperature.mean('time'),
    ax=ax1,
    colorbar_label='Temperature (°C)',
    title='Mean Temperature'
)

# Plot 95th percentile
ax2 = plt.subplot(122, projection=ccrs.PlateCarree())
plot_utils.plot_field(
    temp_95,
    ax=ax2,
    colorbar_label='Temperature (°C)',
    title='95th Percentile Temperature'
)

plt.tight_layout()
plt.show()

## 5. Return Period Analysis

Let's analyze return periods for extreme temperatures.

In [ ]:
# Get temperature values for return period analysis
temp_values = np.linspace(temp_series.min(), temp_series.max(), 100)

# Create return period plot
plt.figure(figsize=(10, 6))
plot_utils.plot_return_period(
    temp_series,
    temp_values,
    ylabel='Temperature (°C)',
    title='Temperature Return Periods'
)
plt.grid(True)
plt.show()

## 6. Physical Consistency

Let's demonstrate some meteorological calculations.

In [ ]:
# Create sample pressure data (simplified for demonstration)
pressure = 1013.25 - 0.1 * ds.latitude

# Calculate potential temperature
theta = met_utils.potential_temperature(
    ds.temperature.isel(time=0),
    pressure
)

# Plot potential temperature
plt.figure(figsize=(10, 6))
plot_utils.plot_field(
    theta,
    colorbar_label='Potential Temperature (K)',
    title='Potential Temperature'
)
plt.show()